# Importing required modules

In [25]:
!pip install pillow
!pip install pdfminer
!pip install pytesseract
!pip install PyPDF2
!pip install docx2txt
!pip install nltk

nltk.download('all')

import pytesseract
from PIL import Image

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

import PyPDF2
import os

import requests
from bs4 import BeautifulSoup as soup
import os

import nltk
import docx2txt
import io
import sys

l=os.listdir()
docx=[i for i in l if '.docx' in i]

# PATH for storing image inside .docx file
path = r"C:\Users\VINU PC\Desktop"

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [53]:
ip=input("Enter the file name:")
iplist=ip.split(".")
if(iplist[1]=='docx'):
    l=getText(ip)
elif(iplist[1]=='pdf'):
    l=convert_pdf_to_txt(ip)
else:
    l=getWeb(ip)
print(l)

Enter the file name:a.pdf


TypeError: __init__() got an unexpected keyword argument 'codec'

# 1. Extracting data from url : 

In [43]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getWeb(url):
    html = urllib.request.urlopen(url).read()
    return text_from_html(html)

# 2. Extracting data from text file

In [9]:
def getText(filename):
    text = docx2txt.process(filename, path) 
    return text

# 3. Extracting data from pdf

In [52]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)



    fp.close()
    device.close()
    text = retstr.getvalue()
    retstr.close()
    text=str(text.strip())
    return text


def OCR(path):
    try:
        text = pytesseract.image_to_string(Image.open(path))
        # print("ocr",text)
        return text
    except Exception as e:
        # print(e)
        return

def image_text(path):
    a=[]
    with open(path,'rb') as pdf_file:
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        number_of_pages = read_pdf.getNumPages()

        page = read_pdf.getPage(number_of_pages-1)

        data=image_text_pagewise(page)
        if data:
            a.append(data)
        '''   
        for page_number in range(number_of_pages):   # use xrange in Py2
            # page = read_pdf.getPage(page_number)
            # page_content = page.extractText()
            # image_text_pagewise(read_pdf,page_number)
            page = read_pdf.getPage(page_number)
            data=image_text_pagewise(page_number,page)
            # a=[]
            if data:
                a.append(data)
        # print("HELLO!@#\n",a)
        '''
    return a

def image_text_pagewise(page0):
    # input1 = PyPDF2.PdfFileReader(open(path, "rb"))
    d=dict()
    cont=[]
    # print(page0)
    if '/XObject' in page0['/Resources']:
        xObject = page0['/Resources']['/XObject'].getObject()
        # print(xObject)
        for obj in xObject:
            if xObject[obj]['/Subtype'] == '/Image':
                size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                # print(size)
                # print(xObject[obj])
                data = xObject[obj]._data
                # if data.endswith(b'~>'):
                #     data = base64.a85decode(data, adobe=True)
                # print(data)
                if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                    mode = "RGB"
                else:
                    mode = "P"
                
                if '/Filter' in xObject[obj]:
                    if xObject[obj]['/Filter'] == '/FlateDecode':
                        data = xObject[obj].getData()
                        img = Image.frombytes(mode, size, data)
                        im=obj[1:] + ".png"
                        img.save(im)
                        t=OCR(im)
                        if t:
                            cont.append(t)
                        os.remove(im)

                    elif xObject[obj]['/Filter'] == '/DCTDecode':
                        im=obj[1:] + ".jpg"
                        img = open(im, "wb")
                        img.write(data)
                        t=OCR(im)
                        # print("OCR T",len(t))
                        if t:
                            cont.append(t)
                        os.remove(im)

                    elif xObject[obj]['/Filter'] == '/JPXDecode':
                        im=obj[1:] + ".jp2"
                        img = open(im, "wb")
                        img.write(data)
                        t=OCR(im)
                        if t:
                            cont.append(t)
                        os.remove(im)
                    elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
                        im=obj[1:] + ".tiff"
                        img = open(im, "wb")
                        img.write(data)
                        t=OCR(im)
                        if t:
                            cont.append(t)
                        os.remove(im)
                else:
                    img = Image.frombytes(mode, size, data)
                    im=obj[1:] + ".png"
                    t=OCR(im)
                    if t:
                        cont.append(t)
                    os.remove(im)
            # else:
            #     print(xObject[obj]['/Subtype'],"No xobj",page0['/Resources'])
            #     size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
            #     print(size)
    # else:
    #     print("No image found.")
    # print("OCR\n\n\\n\n\n",d,"\n\n\\n")
    if cont:
        return cont
    return

def file_extension(l):
    if(len(l)<2):
        return
    d=dict()
    for i in l[1:]:
        file_ext=i.split(".")[-1]
        if(file_ext not in d):
            d[file_ext]=[]
        d[file_ext].append(i);
        d[file_ext]=list(set(d[file_ext]))
    return d


def pdf(file_pdf):
    d=dict()
    a=[]
    for files in file_pdf:
        d[files]=[]
        text_data=convert_pdf_to_txt(files)
        image_data=image_text(files)
        d[files].append(text_data)
        d[files].append(image_data)
        # d[files]=list(set([text_data,image_data]))
    return d

def inp_files(d):
    k=[]
    for file_ext in d:
        if(file_ext=="pdf"):
            k.append(pdf(d[file_ext]))
        elif(file_ext=="html"):
            # k.append("HTML\n\n\\n\n\n\n\n\n\n\n\n\n\n\n")
            print(file_ext,"YET TO BE DONE")
    return k



def display(f,l):
    for i in f["pdf"]:
        k=l[i]
        print("\n\n----FILENAME ---- ",i,"----------------------------------------------------------------------------\n\n")
        print("----------TEXT DATA------------\n\n",k[0])
        if(len(k[1])):
            for j in k[1][0]:
                print("\n\n----------IMAGE DATA ---------------\n\n",j)


def intermediate_display(f,l):
    for i in l:
        display(f,i)

def pdfData(f):
     # Extract_text(sys.argv)
    l=inp_files(f)
    intermediate_display(f,l)
    # print(l)



# Pre processing extracted string

## text normalization - converting all letters to lower case :
## lower case is chosen since number of conversions required is less

In [46]:
#Here we tokenize the string by words and then perform stop word removal anlog with punctuation removal
#remove whitespaces in the string
l=l.lower()
l=l.strip()

w = word_tokenize(l)
sw = stopwords.words('english')
sw.extend(list(".,/()[]{}|~:?@<=>;#$%&!+-*/'"))
result = [i for i in w if i not in sw]
#result

In [47]:
#stemming : it just simply removes morphological inflections not very good.
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
for p in result:
    print(stemmer.stem(p))

ashoka
wikipedia
free
encyclopedia
jump
navig
jump
search
3rd
emperor
maurya
dynasti
patron
buddhism
``
aśoka
''
``
asoka
''
redirect
use
see
ashoka
disambigu
‹
templat
infobox
royalti
consid
merg
›
ashoka
c.
1st
centuri
bce/c
relief
sanchi
show
ashoka
chariot
visit
naga
ramagrama
1
2
3rd
mauryan
emperor
reign
c.
268
–
c.
232
bce
3
coron
268
bce
3
predecessor
bindusara
successor
dasharatha
born
c.
304
bce
pataliputra
mauryan
empir
adjac
present-day
patna
bihar
india
die
232
bce
pataliputra
modern-day
patna
bihar
india
spous
karuvaki
inscript
tishyaraksha
sri
lankan
north
indian
tradit
padmavati
north
indian
tradit
asandhimitra
sri
lankan
tradit
devi
sri
lankan
tradit
issu
tivala
inscript
kunala
north
indian
tradit
mahendra
sri
lankan
tradit
sanghamitra
sri
lankan
tradit
jalauka
rajatarangini
charumati
dynasti
maurya
father
bindusara
mother
subhadrangi
north
indian
tradit
dharma
sri
lankan
tradit
see
mother
ashoka
religion
buddhism
maurya
empir
322–180
bce
chandragupta
322–297
bce
bindu

birth
remov
sorrow
32
name
priyadasi
associ
ashoka
3rd-4th
centuri
ce
dipavamsa
33
34
term
liter
mean
``
regard
amiabl
''
``
graciou
mien
''
sanskrit
priya-darshi
may
regnal
name
adopt
ashoka
35
ashoka
's
inscript
mention
titl
devanampiya
sanskrit
devanampriya
``
belov
god
''
identif
devanampiya
ashoka
person
establish
maski
gujarra
inscript
use
term
king
36
37
titl
adopt
king
includ
contemporari
king
devanampiya
tissa
anuradhapura
ashoka
's
descend
dasharatha
maurya
38
earli
life
ashoka
's
inscript
describ
earli
life
much
inform
topic
come
apocryph
legend
written
hundr
year
39
legend
includ
obvious
fictiti
detail
narr
ashoka
's
past
live
includ
plausibl
histor
detail
ashoka
's
period
39
40
date
major
rock
edict
no.13
ashoka
mention
greek
king
antiochu
ptolemi
antigonu
maga
alexand
name
recipi
teach
exact
date
ashoka
's
birth
certain
extant
contemporari
indian
text
record
detail
known
live
3rd
centuri
bce
inscript
mention
sever
contemporari
ruler
whose
date
known
certainti
antiochu
ii


half
year
make
much
progress
past
year
drawn
closer
sangha
becam
ardent
follow
93
war
accord
ashoka
's
major
rock
edict
13
conquer
kalinga
8
year
ascens
throne
edict
state
conquest
kalinga
100,000
men
anim
kill
action
mani
time
number
``
perish
''
150,000
men
anim
carri
away
kalinga
captiv
ashoka
state
repent
suffer
caus
devot
practic
propag
dharma
96
proclaim
consid
slaughter
death
deport
caus
conquest
countri
pain
deplor
consid
suffer
caus
religi
peopl
household
even
deplor
96
edict
found
inscrib
sever
place
includ
erragudi
girnar
kalsi
maneshra
shahbazgarhi
kandahar
97
howev
omit
ashoka
's
inscript
found
kalinga
region
rock
edict
13
14
replac
two
separ
edict
make
mention
ashoka
's
remors
possibl
ashoka
consid
polit
appropri
make
confess
peopl
kalinga
98
anoth
possibl
kalinga
war
consequ
describ
ashoka
's
rock
edict
``
imaginari
real
''
descript
meant
impress
far
remov
scene
thu
unabl
verifi
accuraci
99
ancient
sourc
mention
militari
activ
ashoka
although
16th
centuri
writer
taranath

centuri
ce
relief
bear
inscript
``
rāya
asoko
''
𑀭𑀸𑀬
𑀅𑀲𑁄𑀓𑁄
``
king
ashoka
''
brahmi
script
depict
king
queen
two
attend
bear
fly-whisk
one
attend
bear
umbrella
144
2
emperor
ashoka
queen
deer
park
sanchi
relief
2
queen
variou
sourc
mention
five
consort
ashoka
devi
vedisa-mahadevi-shakyakumari
karuvaki
asandhimitra
pali
asandhimitta
padmavati
tishyarakshita
pali
tissarakkha
145
kaurvaki
queen
ashoka
known
inscript
mention
edict
inscrib
pillar
allahabad
inscript
name
mother
princ
tivara
order
royal
offic
mahamatta
record
religi
charit
donat
80
accord
one
theori
tishyarakshita
regnal
name
kaurvaki
80
accord
mahavamsa
ashoka
's
chief
queen
asandhimitta
die
four
year
80
state
born
ashoka
's
queen
previou
life
direct
pratyekabuddha
honey
merchant
later
reborn
ashoka
146
later
text
also
state
addit
gave
pratyekabuddha
piec
cloth
made
147
text
includ
dasavatthuppakarana
so-cal
cambodian
extend
mahavamsa
possibl
9th–10th
centuri
trai
bhumi
katha
15th
centuri
147
text
narrat
anoth
stori
one
day


ambassador
hellenist
king
india
well
düta
sent
ashoka
186
dionysiu
report
greek
ambassador
court
ashoka
sent
ptolemi
ii
philadelphu
188
mention
edict
ashoka
recipi
buddhist
proselyt
ashoka
hellenist
philosoph
hegesia
cyren
probabl
live
rule
king
maga
one
suppos
recipi
buddhist
emissari
asoka
sometim
thought
influenc
buddhist
teach
189
greek
india
even
seem
play
activ
role
propag
buddhism
emissari
ashoka
dharmaraksita
describ
pali
sourc
lead
greek
yona
buddhist
monk
activ
spread
buddhism
mahavamsa
xii
190
greek
yavana
may
play
administr
role
territori
rule
ashoka
girnar
inscript
rudradaman
record
rule
ashoka
yavana
governor
charg
area
girnar
gujarat
mention
role
construct
water
reservoir
191
thought
ashoka
's
palac
patna
model
achaemenid
palac
persepoli
192
legend
past
live
buddhist
legend
mention
stori
ashoka
's
past
live
accord
mahavamsa
stori
ashoka
nigrodha
devnampiya
tissa
brother
previou
life
life
pratyekabuddha
look
honey
cure
anoth
sick
pratyekabuddha
woman
direct
honey
shop
own

^
strong
1989
p.
12
^
b
c
e
f
g
h
strong
1995
p.
143
^
b
c
e
strong
1995
p.
144
^
strong
1995
pp
152–154
^
b
c
strong
1995
p.
155
^
strong
1995
pp
154–157
^
thapar
1961
p.
11
^
thapar
1995
p.
15
^
b
thapar
1961
p.
9
^
gurug
review
1995
pp
185-188
^
b
c
e
f
g
h
beckwith
christoph
2017
greek
buddha
pyrrho
's
encount
earli
buddhism
central
asia
princeton
univers
press
pp
226–250
isbn
978-0-691-17632-1
^
strong
1989
p.
205
^
allen
charl
2012
ashoka
search
india
's
lost
emperor
littl
brown
book
group
p.
79.
isbn
9781408703885
^
dîpavaṃsa
ancient
buddhist
histor
record
william
norgat
1879.
pp
147
–148
^
thapar
1961
pp
226–227
^
strong
1989
p.
11
^
lahiri
2015
p.
129
^
thapar
1961
p.
226
^
b
lahiri
2015
p.
27
^
b
c
singh
2008
p.
332
^
b
lahiri
2015
p.
25
^
lahiri
2015
p.
24
^
lahiri
2015
p.
26
^
b
thapar
1961
p.
13
^
b
strong
1989
p.
204
^
b
thapar
1961
pp
25–26
^
strong
1989
pp
204–205
^
lahiri
2015
p.
323
''
ashokavadana
ashoka
's
mother
name
''
^
lahiri
2015
p.
31
^
b
gurug
1993
p.
19
^
b


39
^
handbuch
der
orientalistik
de
kurt
a.
behrendt
p.
39
v
e
topic
buddhism
outlin
glossari
index
foundat
four
nobl
truth
three
jewel
buddha
dharma
sangha
nobl
eightfold
path
nirvana
middl
way
buddha
tathāgata
birthday
four
sight
physic
characterist
footprint
relic
iconographi
lao
thailand
film
miracl
famili
suddhodāna
father
māyā
mother
mahapajapati
gotamī
aunt
adopt
mother
yaśodharā
wife
rāhula
son
ānanda
cousin
devadatta
cousin
place
buddha
stay
buddha
world
religion
bodhisattva
avalokiteśvara
guanyin
mañjuśrī
mahāsthāmaprāpta
ākāśagarbha
kṣitigarbha
samantabhadra
vajrapāṇi
skanda
tārā
metteyya/maitreya
discipl
kaundinya
assaji
sāriputta
mahamoggallāna
ānanda
mahākassapa
aṅgulimāla
anuruddha
mahākaccana
nanda
subhūti
punna
upāli
mahapajapati
gotamī
khema
uppalavanna
asita
channa
yasa
key
concept
avidyā
ignor
bardo
bodhicitta
buddha-natur
dhamma
theori
dharma
enlighten
five
hindranc
indriya
karma
klesha
mindstream
parinirvana
pratītyasamutpāda
rebirth
saṃsāra
saṅkhāra
skandha
śūnyat

In [48]:
#lemmatization : it is better than stemming since it uses lexical knowledge bases to get corrrect base form of the words
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
#for w in result:
#    print(w, wordnet_lemmatizer.lemmatize(w))
res_lem=[]
for w in result:
    res_lem.append(wordnet_lemmatizer.lemmatize(w))

In [49]:
res_lem # contains the lemmatized words from the result string.

['ashoka',
 'wikipedia',
 'free',
 'encyclopedia',
 'jump',
 'navigation',
 'jump',
 'search',
 '3rd',
 'emperor',
 'maurya',
 'dynasty',
 'patron',
 'buddhism',
 '``',
 'aśoka',
 "''",
 '``',
 'asoka',
 "''",
 'redirect',
 'us',
 'see',
 'ashoka',
 'disambiguation',
 '‹',
 'template',
 'infobox',
 'royalty',
 'considered',
 'merging',
 '›',
 'ashoka',
 'c.',
 '1st',
 'century',
 'bce/ce',
 'relief',
 'sanchi',
 'showing',
 'ashoka',
 'chariot',
 'visiting',
 'naga',
 'ramagrama',
 '1',
 '2',
 '3rd',
 'mauryan',
 'emperor',
 'reign',
 'c.',
 '268',
 '–',
 'c.',
 '232',
 'bce',
 '3',
 'coronation',
 '268',
 'bce',
 '3',
 'predecessor',
 'bindusara',
 'successor',
 'dasharatha',
 'born',
 'c.',
 '304',
 'bce',
 'pataliputra',
 'mauryan',
 'empire',
 'adjacent',
 'present-day',
 'patna',
 'bihar',
 'india',
 'died',
 '232',
 'bce',
 'pataliputra',
 'modern-day',
 'patna',
 'bihar',
 'india',
 'spouse',
 'karuvaki',
 'inscription',
 'tishyaraksha',
 'sri',
 'lankan',
 'north',
 'indian',
 

In [50]:
from nltk import pos_tag
print(pos_tag(res_lem))

[('ashoka', 'JJ'), ('wikipedia', 'WRB'), ('free', 'JJ'), ('encyclopedia', 'NN'), ('jump', 'NN'), ('navigation', 'NN'), ('jump', 'NN'), ('search', 'NN'), ('3rd', 'CD'), ('emperor', 'NN'), ('maurya', 'NN'), ('dynasty', 'JJ'), ('patron', 'NN'), ('buddhism', 'NN'), ('``', '``'), ('aśoka', 'NN'), ("''", "''"), ('``', '``'), ('asoka', 'NN'), ("''", "''"), ('redirect', 'VBP'), ('us', 'PRP'), ('see', 'VBP'), ('ashoka', 'JJ'), ('disambiguation', 'NN'), ('‹', 'NNP'), ('template', 'NN'), ('infobox', 'NN'), ('royalty', 'NN'), ('considered', 'VBN'), ('merging', 'VBG'), ('›', 'JJ'), ('ashoka', 'JJ'), ('c.', 'NN'), ('1st', 'JJ'), ('century', 'NN'), ('bce/ce', 'NN'), ('relief', 'NN'), ('sanchi', 'JJ'), ('showing', 'VBG'), ('ashoka', 'JJ'), ('chariot', 'NN'), ('visiting', 'VBG'), ('naga', 'JJ'), ('ramagrama', 'NN'), ('1', 'CD'), ('2', 'CD'), ('3rd', 'CD'), ('mauryan', 'JJ'), ('emperor', 'NN'), ('reign', 'NN'), ('c.', 'VBD'), ('268', 'CD'), ('–', 'NNP'), ('c.', 'NN'), ('232', 'CD'), ('bce', 'NN'), ('3',